1. Create a flat table per each order (flat – for every order bring every relevant information). For example: users registration, country, category etc. Save it to csv or excel file. PLEASE  validate the data.

2. Read the extra text file that can be find here to your current excel

In [37]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [39]:
db_config = {'user': 'practicum_student',         # username             
                      'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password             
                      'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',       #host URL      
                      'port': 6432,              # connection port             
                      'db': 'data-analyst-web-store'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [40]:
def queryResult(q):
    return pd.io.sql.read_sql(q, con = engine)

In [116]:

query1 = '''
            select
                    finale_table.order_id, finale_table.date,
                    Registration,
                    finale_table.campaign_id, finale_table.source,  finale_table.camp_start_date , finale_table.camp_end_date ,
                    finale_table.customer_id, finale_table.membership, finale_table.gender,
                    finale_table.country_id, finale_table.symbol, finale_table.country,
                    finale_table.ab_test_id, finale_table.id, finale_table.group,
                    finale_table.description, finale_table.ab_start_date, finale_table.ab_end_date,
                    finale_table.amount,
                    finale_table.item_id, finale_table.price, finale_table.item,
                    finale_table.vendor_id, finale_table.vendor,
                    finale_table.category_id, finale_table.category, finale_table.us_ship, finale_table.other_ship
            from
            (
                select  
                    orders.order_id, orders.date,
                    Registration,
                    campaign.campaign_id, campaign.source,  campaign.start_date as camp_start_date, campaign.end_date as camp_end_date,
                    customer_info.customer_id, customer_info.membership, customer_info.gender,
                    countries.country_id, countries.symbol, countries.country,
                    ab_allocation.ab_test_id, ab_allocation.id, ab_allocation.group,
                    ab_info.description, ab_info.start_date as ab_start_date, ab_info.end_date as ab_end_date,
                    order_details.amount,
                    items.item_id, items.price, items.item,
                    vendors.vendor_id, vendors.vendor,
                    categories.category_id, categories.category, categories.us_ship, categories.other_ship,
                    row_number() over (partition by orders.order_id order BY (select null)) as duprank
                from 
                    orders 
                    left join registration on orders.customer_id = registration.customer_id
                    left join campaign on campaign.campaign_id = registration.campaign_id
                    left join customer_info on customer_info.customer_id = orders.customer_id
                    left join countries on countries.country_id = customer_info.country_id
                    left join ab_allocation on ab_allocation.customer_id = orders.customer_id
                    left join ab_info on ab_info.ab_test_id = ab_allocation.ab_test_id
                    left join order_details on order_details.order_id = orders.order_id
                    left join items on items.item_id = order_details.item_id
                    left join vendors on vendors.vendor_id = items.vendor_id
                    left join categories on categories.category_id = items.category_id
            ) as finale_table
            where duprank = 1 and "id" is not null 
         '''


df = queryResult(query1)

display('duplicated: ',df[df.duplicated()])
display('na:', df[df.isna().any(axis=1)])
display('no filter: ',df.head())

'duplicated: '

,order_id,date,registration,campaign_id,source,camp_start_date,camp_end_date,customer_id,membership,gender,...,amount,item_id,price,item,vendor_id,vendor,category_id,category,us_ship,other_ship


'na:'

,order_id,date,registration,campaign_id,source,camp_start_date,camp_end_date,customer_id,membership,gender,...,amount,item_id,price,item,vendor_id,vendor,category_id,category,us_ship,other_ship


'no filter: '

,order_id,date,registration,campaign_id,source,camp_start_date,camp_end_date,customer_id,membership,gender,...,amount,item_id,price,item,vendor_id,vendor,category_id,category,us_ship,other_ship
0,1,2019-10-02,"(214,""2019-06-15 00:00:00"",3)",3,Google Ads,2019-05-30,2019-12-31,214,yes,Male,...,3,156,13.99,Wilson Official Size NFL Ball American Football,21,vhbw-online,5,Sporting Goods,14.99,19.99
1,2,2019-10-02,"(459,""2019-06-29 00:00:00"",2)",2,Facebook,2019-05-30,2019-12-31,459,yes,Female,...,2,52,84.99,GK61s Mechanical Gaming Keyboard,20,trainz,9,Gaming,9.99,14.99
2,3,2019-10-02,"(489,""2019-05-29 00:00:00"",1)",1,YouTube,2019-05-30,2019-12-31,489,no,Male,...,1,116,21.95,Revlon Super Lusrous 3 piece,24,beautyo,10,Beauty Care,9.99,19.99
3,4,2019-10-03,"(127,""2019-05-02 00:00:00"",2)",2,Facebook,2019-05-30,2019-12-31,127,yes,Female,...,3,30,95.95,Cross Trainer Air Walker Glider Home Gym,12,lumo-shop,5,Sporting Goods,14.99,19.99
4,5,2019-10-03,"(174,""2019-09-24 00:00:00"",3)",3,Google Ads,2019-05-30,2019-12-31,174,no,Male,...,3,136,29.95,Star Wars: the Old Republic - Legends Ser.: Fa...,6,senior pac-man,3,Books,9.99,14.99


In [117]:
# export to excel 
df.to_excel("online_shop.xlsx")

In [119]:
# convert txt file to excel

# make sure to install openpyxl - pip install openpyxl
data = pd.read_csv("expense.txt", sep='\t')
print(data)
data.to_excel("expense.xlsx")

    campaign_id      source  start_date    end_date  cost_per_customer
0             1     YouTube   5/30/2019  12/31/2019                 12
1             2    Facebook   5/30/2019  12/31/2019                 15
2             3  Google Ads   5/30/2019  12/31/2019                 10
3             4     YouTube    1/1/2020   1/31/2020                  9
4             5    Facebook    1/1/2020   3/31/2020                 12
5             6  Google Ads    2/1/2020   2/28/2020                 13
6             7     YouTube   3/25/2020   6/30/2020                 15
7             8    Facebook   6/18/2020   8/30/2020                 16
8             9  Google Ads    5/1/2020    9/1/2020                 18
9            10     YouTube   8/12/2020  12/31/2020                 19
10           11    Facebook  11/13/2020   3/10/2021                 13
11           12     YouTube   2/23/2021   6/30/2021                 11
12           13    Facebook    4/1/2021   7/12/2021                 10
13    